In [0]:
import os
import openai
import logging
import dataiku
import resource
import pandas as pd

from time import time
from utils_prompt import prepare_excel_for_pdf, convert_office_to_pdf, extract_image_pdf

In [0]:
logging.getLogger('openai').setLevel(logging.WARNING)
logging.getLogger('httpx').setLevel(logging.WARNING)

client = dataiku.api_client()
auth_info = client.get_auth_info(with_secrets=True)

for secret in auth_info['secrets']:
    if secret['key'] == 'neura-gpt-test-key':
        API_KEY = secret['value']
        break
        
openai.api_type = 'azure'
openai.azure_endpoint = 'https://iapi-test.merck.com/gpt/libsupport'
openai.api_version = '2025-03-01-preview'
openai.api_key = API_KEY

model_name = 'gpt-4o-2024-11-20'
price_per_mil_token = (2.5, 10)

print("test")

In [0]:
folder_pref = '/tmp/auto_soe'

environ = os.environ.copy()
environ['XDG_RUNTIME_DIR'] = folder_pref

os.makedirs(folder_pref, exist_ok=True)

sources = [
    dataiku.Folder('jpYSb1ya'),
    dataiku.Folder('Ms4f1qz8')
]

all_entries = []
for fobj in sources:
    for p in fobj.list_paths_in_partition():
        all_entries.append((fobj, p))
            
print(all_entries)      
